In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import folium
from folium.plugins import HeatMap
import requests
import geopandas as gpd


In [9]:
fichier_geojson = '/Users/aurelienbrame/code/CyclingFacilities/raw_data/Pistes/france-20220710.geojson'

data_pistes = gpd.read_file(fichier_geojson)


In [10]:
data_pistes.head()


,id_local,id_osm,num_iti,code_com_d,ame_d,regime_d,sens_d,largeur_d,local_d,statut_d,...,revet_g,access_ame,date_maj,trafic_vit,lumiere,d_service,source,project_c,ref_geo,geometry
0,geovelo_377971705_31254,377971705,None,31254,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,LISSE,None,2020-08-10,5.0,True,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (1.52869 43.53102, 1.52760 43.53166)"
1,geovelo_568517126_24098,568517126,None,24098,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2020-06-04,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.72652 45.21236, 0.72615 45.21294..."
2,geovelo_82213402_37214,82213402,None,37214,PISTE CYCLABLE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2010-10-19,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.66715 47.42967, 0.66709 47.42969)"
3,geovelo_38861770_37261,38861770,EV6,37261,VOIE VERTE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2022-04-07,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.69789 47.36713, 0.69797 47.36715)"
4,geovelo_184154243_37208,184154243,None,37208,VOIE VERTE,AUTRE,UNIDIRECTIONNEL,NaN,None,EN SERVICE,...,None,None,2017-02-15,5.0,None,None,Les contributeurs OpenStreetmap,4326,OpenStreetmap,"LINESTRING (0.73012 47.34787, 0.73015 47.34783)"


In [26]:
data_pistes.columns


Index(['id_local', 'id_osm', 'num_iti', 'code_com_d', 'ame_d', 'regime_d',
       'sens_d', 'largeur_d', 'local_d', 'statut_d', 'revet_d', 'code_com_g',
       'ame_g', 'regime_g', 'sens_g', 'largeur_g', 'local_g', 'statut_g',
       'revet_g', 'access_ame', 'date_maj', 'trafic_vit', 'lumiere',
       'd_service', 'source', 'project_c', 'ref_geo', 'geometry'],
      dtype='object')

In [12]:
data_pistes.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 259030 entries, 0 to 259029
Data columns (total 28 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   id_local    259030 non-null  object  
 1   id_osm      259030 non-null  object  
 2   num_iti     30594 non-null   object  
 3   code_com_d  258929 non-null  object  
 4   ame_d       259021 non-null  object  
 5   regime_d    221506 non-null  object  
 6   sens_d      258969 non-null  object  
 7   largeur_d   11071 non-null   float64 
 8   local_d     80287 non-null   object  
 9   statut_d    259030 non-null  object  
 10  revet_d     146047 non-null  object  
 11  code_com_g  258925 non-null  object  
 12  ame_g       259030 non-null  object  
 13  regime_g    177430 non-null  object  
 14  sens_g      207286 non-null  object  
 15  largeur_g   10019 non-null   float64 
 16  local_g     78539 non-null   object  
 17  statut_g    259030 non-null  object  
 18  revet_g     1460

In [24]:
data_pistes['statut_d'].value_counts()


EN SERVICE    255967
PROVISOIRE      3063
Name: statut_d, dtype: int64

### Data Accidents

In [2]:
fichier_accident = '/Users/aurelienbrame/Desktop/Projet Mobilité/Mobilité/Data/Accidents/accidentsVelo-full.csv'
data_accident = pd.read_csv(fichier_accident)
data_accident.head()


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_4685/3899721259.py:2: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  data_accident = pd.read_csv(fichier_accident)


,Num_Acc,date,an,mois,jour,hrmn,dep,com,lat,long,...,equipement,obs,obsm,choc,manv,vehiculeid,typevehicules,manoeuvehicules,numVehicules,_infos_commune.code_epci
0,200500000030,2005-01-13,2005,janvier,jeudi,19:45,62,62331,50.300,2.840,...,0,0.0,2.0,8.0,11.0,200500000030B02,18,17,1.0,200033579.0
1,200500000034,2005-01-19,2005,janvier,mercredi,10:45,62,62022,0.000,0.000,...,0,0.0,2.0,1.0,1.0,200500000034B02,10,15,1.0,200018083.0
2,200500000078,2005-01-26,2005,janvier,mercredi,13:15,02,02173,0.000,0.000,...,2,0.0,2.0,1.0,1.0,200500000078B02,7,15,1.0,200071785.0
3,200500000093,2005-01-03,2005,janvier,lundi,13:30,02,02810,49.255,3.094,...,0,0.0,2.0,3.0,21.0,200500000093B02,7,21,1.0,200071991.0
4,200500000170,2005-01-29,2005,janvier,samedi,18:30,76,76196,0.000,0.000,...,9,0.0,2.0,4.0,2.0,200500000170A01,10,2,1.0,200084952.0


In [8]:
# Situation accident par type de chaussée / Pertinent
data_accident['situ'].value_counts()


 1.0    62033
 5.0     5589
 0.0     3674
 4.0     1562
 3.0      948
 6.0      243
 2.0      106
 8.0      105
-1.0        4
Name: situ, dtype: int64

### Data Flux

In [30]:
fichier_compteurs = '/Users/aurelienbrame/code/CyclingFacilities/raw_data/Flux/comptage-velo-compteurs.geojson'
fichier_compteurs_data = '/Users/aurelienbrame/code/CyclingFacilities/raw_data/Flux/comptage-velo-donnees-compteurs.geojson'


In [31]:
data_flux_compteurs = gpd.read_file(fichier_compteurs)


In [33]:
data_flux_compteurs_data = gpd.read_file(fichier_compteurs_data)


In [34]:
data_flux_compteurs_data.head()


,id_compteur,nom_compteur,id,name,sum_counts,date,installation_date,url_photos_n1,coordinates,counter,photos,test_lien_vers_photos_du_site_de_comptage_,id_photo_1,url_sites,type_dimage,mois_annee_comptage,geometry
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,3,2022-11-01 05:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,2,2022-11-01 03:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)
2,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,5,2022-11-01 04:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)
3,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,8,2022-11-01 06:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)
4,100003096-353242251,97 avenue Denfert Rochereau SO-NE,100003096,97 avenue Denfert Rochereau,60,2022-11-01 08:00:00+00:00,2012-02-22,https://filer.eco-counter-tools.com/file/10/6d...,None,X2H20012081,https://filer.eco-counter-tools.com/file/10/6d...,https://filer.eco-counter-tools.com/file/10/6d...,https:,https://www.eco-visio.net/Photos/100003096,jpg,2022-11,POINT (2.33314 48.83504)


In [36]:
data_flux_compteurs_data.shape


(926954, 17)

In [38]:
data_flux_compteurs_data.columns


Index(['id_compteur', 'nom_compteur', 'id', 'name', 'sum_counts', 'date',
       'installation_date', 'url_photos_n1', 'coordinates', 'counter',
       'photos', 'test_lien_vers_photos_du_site_de_comptage_', 'id_photo_1',
       'url_sites', 'type_dimage', 'mois_annee_comptage', 'geometry'],
      dtype='object')

In [ ]:
data_flux_compteurs_data.drop('url_photos_n1', axis=1, inplace=True)


In [40]:
data_flux_compteurs_data.drop('type_dimage', axis=1, inplace=True)


In [47]:
data_flux_compteurs_data.drop('photos', axis=1, inplace=True)


In [49]:
data_flux_compteurs_data.drop('id_photo_1', axis=1, inplace=True)


In [54]:
data_flux_compteurs_data.drop('test_lien_vers_photos_du_site_de_comptage_', axis=1, inplace=True)


In [56]:
data_flux_compteurs_data.drop('url_sites', axis=1, inplace=True)


In [95]:
data_flux_compteurs_data.drop('coordinates', axis=1, inplace=True)


In [97]:
data_flux_compteurs_data.drop('name', axis=1, inplace=True)


In [99]:
data_flux_compteurs_data.drop('id', axis=1, inplace=True)


In [100]:
data_flux_compteurs_data.head()


,id_compteur,nom_compteur,sum_counts,date,installation_date,counter,mois_annee_comptage,geometry
0,100003096-353242251,97 avenue Denfert Rochereau SO-NE,3,2022-11-01 05:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)
1,100003096-353242251,97 avenue Denfert Rochereau SO-NE,2,2022-11-01 03:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)
2,100003096-353242251,97 avenue Denfert Rochereau SO-NE,5,2022-11-01 04:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)
3,100003096-353242251,97 avenue Denfert Rochereau SO-NE,8,2022-11-01 06:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)
4,100003096-353242251,97 avenue Denfert Rochereau SO-NE,60,2022-11-01 08:00:00+00:00,2012-02-22,X2H20012081,2022-11,POINT (2.33314 48.83504)


In [102]:
data_flux_months = pd.to_datetime(data_flux_compteurs_data['date'])
counts_by_month = data_flux_compteurs_data.groupby(data_flux_compteurs_data['date'].dt.to_period('M')).sum()
counts_by_month


/Users/aurelienbrame/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/pandas/core/arrays/datetimes.py:1162: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


,sum_counts
date,
2022-03,0
2022-04,0
2022-07,0
2022-08,5
2022-11,4828098
2022-12,3290438
2023-01,4363337
2023-02,4778604
2023-03,5546875


### Test loop data 

In [3]:
import yaml


In [6]:
chemin_fichier_yml = '../config.yml'
chemin_dossier = '../raw_data/Accidents/year_data'


with open (chemin_fichier_yml, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    rename_config = config.get('rename')

for old_name, new_name in rename_config.items():
    chemin_ancien_fichier = os.path.join(chemin_dossier, old_name)
    chemin_nouveau_fichier = os.path.join(chemin_dossier, new_name)

    if os.path.exists(chemin_ancien_fichier):
        os.rename(chemin_ancien_fichier, chemin_nouveau_fichier)
        print(f"Fichier renommé : {old_name} -> {new_name}")
    else:
        print(f"Fichier non trouvé : {old_name}")


Fichier non trouvé : caracteristiques-2017.csv
Fichier renommé : caracteristiques-2018.csv -> caracteristiques_2018.csv
Fichier non trouvé : caracteristiques-2019.csv
Fichier non trouvé : caracteristiques-2020.csv
Fichier non trouvé : carcteristiques-2021.csv
Fichier non trouvé : carcteristiques-2022.csv
Fichier non trouvé : lieux-2017.csv
Fichier non trouvé : lieux-2018.csv
Fichier non trouvé : lieux-2019.csv
Fichier non trouvé : lieux-2020.csv
Fichier non trouvé : lieux-2021.csv
Fichier non trouvé : lieux-2022.csv
Fichier non trouvé : usagers-2017.csv
Fichier non trouvé : usagers-2018.csv
Fichier non trouvé : usagers-2019.csv
Fichier non trouvé : usagers-2020.csv
Fichier non trouvé : usagers-2021.csv
Fichier non trouvé : usagers-2022.csv
Fichier non trouvé : vehicules-2017.csv
Fichier non trouvé : vehicules-2018.csv
Fichier non trouvé : vehicules-2019.csv
Fichier non trouvé : vehicules-2020.csv
Fichier non trouvé : vehicules-2021.csv
Fichier non trouvé : vehicules-2022.csv


In [34]:
chemin_fichier_yml = '../config.yml'


with open(chemin_fichier_yml, 'r') as f:
    config = yaml.safe_load(f)
    config_sep = config.get('sep')
    config_encoding = config.get('encoding')

chemin_dossier = '../raw_data/Accidents/year_data'

df_concat = pd.DataFrame()

csv_files = [file for file in os.listdir(chemin_dossier) if file.endswith('.csv')]
for file in csv_files:
    chemin_fichier = os.path.join(chemin_dossier, file)

    if file in config_sep:
        sep = config_sep[file]
    else:
        sep = ','

    if file in config_encoding:
        encoding = config_encoding[file]
    else:
        encoding = 'utf-8'

    df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)

    df_concat = pd.concat([df_concat, df1])


lieux_2017.csv
utf-8
vehicules_2018.csv
utf-8


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_4685/3963624081.py:28: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)


usagers_2006.csv
utf-8
usagers_2012.csv
utf-8
usagers_2013.csv
utf-8
usagers_2007.csv
utf-8
vehicules_2019.csv
utf-8
lieux_2016.csv
utf-8


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_4685/3963624081.py:28: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)


lieux_2014.csv
utf-8
usagers_2011.csv
utf-8
usagers_2005.csv
utf-8
usagers_2010.csv
utf-8
lieux_2015.csv
utf-8
lieux_2011.csv
utf-8
lieux_2005.csv
utf-8
vehicules_2022.csv
utf-8
usagers_2014.csv
utf-8
usagers_2015.csv
utf-8
lieux_2010.csv
utf-8
lieux_2006.csv
utf-8
lieux_2012.csv
utf-8
vehicules_2009.csv
utf-8
vehicules_2021.csv
utf-8
usagers_2017.csv
utf-8
usagers_2016.csv
utf-8
vehicules_2020.csv
utf-8
vehicules_2008.csv
utf-8
lieux_2013.csv
utf-8
lieux_2007.csv
utf-8
caracteristiques_2018.csv
ISO-8859-1
caracteristiques_2019.csv
utf-8
caracteristiques_2022.csv
utf-8
caracteristiques_2009.csv
utf-8


...

caracteristiques_2021.csv
utf-8
caracteristiques_2020.csv
utf-8
caracteristiques_2008.csv
ISO-8859-1
caracteristiques_2011.csv
ISO-8859-1
caracteristiques_2005.csv
ISO-8859-1
caracteristiques_2010.csv
ISO-8859-1
caracteristiques_2006.csv
ISO-8859-1
caracteristiques_2012.csv
ISO-8859-1
caracteristiques_2013.csv
ISO-8859-1
caracteristiques_2007.csv
ISO-8859-1
caracteristiques_2017.csv
ISO-8859-1
caracteristiques_2016.csv
ISO-8859-1
caracteristiques_2014.csv
ISO-8859-1
caracteristiques_2015.csv
ISO-8859-1
lieux_2022.csv
utf-8


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_4685/3963624081.py:28: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)


vehicules_2005.csv
utf-8
vehicules_2011.csv
utf-8
vehicules_2010.csv
utf-8
lieux_2009.csv
utf-8
lieux_2021.csv
utf-8
vehicules_2012.csv
utf-8
vehicules_2006.csv
utf-8
usagers_2018.csv
utf-8
usagers_2019.csv
utf-8
vehicules_2007.csv
utf-8
vehicules_2013.csv
utf-8
lieux_2020.csv
utf-8
lieux_2008.csv
utf-8
lieux_2018.csv
utf-8


/var/folders/jg/x3y4bp6n6071zs7rycbnfsyc0000gn/T/ipykernel_4685/3963624081.py:28: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(chemin_fichier, sep=sep, encoding=encoding)


vehicules_2017.csv
utf-8
usagers_2021.csv
utf-8
usagers_2009.csv
utf-8
usagers_2008.csv
utf-8
usagers_2020.csv
utf-8
vehicules_2016.csv
utf-8
lieux_2019.csv
utf-8
vehicules_2014.csv
utf-8
usagers_2022.csv
utf-8
vehicules_2015.csv
utf-8


In [35]:
df_concat.shape


(6999518, 59)

In [36]:
df_concat.head()


,Num_Acc,catr,voie,v1,v2,circ,nbv,pr,pr1,vosp,...,gps,lat,long,dep,Accident_Id,vma,secu1,secu2,secu3,id_usager
0,2.017000e+11,3.0,39,NaN,NaN,2.0,2.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.017000e+11,3.0,39,NaN,NaN,2.0,2.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.017000e+11,3.0,39,NaN,NaN,2.0,2.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.017000e+11,3.0,39,NaN,D,2.0,2.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2.017000e+11,3.0,39,NaN,NaN,2.0,2.0,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df_concat.columns


Index(['Num_Acc', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'pr', 'pr1',
       'vosp', 'prof', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ',
       'env1', 'senc', 'catv', 'occutc', 'obs', 'obsm', 'choc', 'manv',
       'num_veh', 'place', 'catu', 'grav', 'sexe', 'trajet', 'secu', 'locp',
       'actp', 'etatp', 'an_nais', 'id_vehicule', 'motor', 'an', 'mois',
       'jour', 'hrmn', 'lum', 'agg', 'int', 'atm', 'col', 'com', 'adr', 'gps',
       'lat', 'long', 'dep', 'Accident_Id', 'vma', 'secu1', 'secu2', 'secu3',
       'id_usager'],
      dtype='object')